# Optimisations



## De la nécessité d'éviter les bloucles autant que possible: Le Temps, cette ressource précieuse

Avant de commencer, j'aimerais débunker une croyance assez répandue, même au sein des devs, selon laquelle python serait, pour des raisons structurelles et inaccessibles, lent lourd et inefficient pour des applications à taille réelle. Deux niveau de réponses

Premièrement, ce problème n'est pas propre à Python mais concerne l'ensemble des langages de haut degré d'abstraction (trad "vous voulez du rapide, codez en C")

Deuxièmement, le problème ne provient pas tant du langage en lui-même que du fait que les opérations précises de l'interpréteurs ne nous étant pas accessibles, deux <b>implémentations différentes</b> de la même tâche peuvent demander un temps assez différent alors même qu'elle paraissent "conceptuellement identiques". 

Prenons un exemple en nous dotant tout d'abord d'un moyen de benchmarker nos opérations avant de chercher à optimiser notre code

In [2]:
from time import time
import math
import numpy as np

def Benchmark(fonction): 
    start = time()
    fonction
    end = time()
    return (end - start)

In [3]:
dataTest = np.random.rand(600000,3) # on génère 600000*3 points de données x,y,z (vecteur 2-D)
dataTestAsList = dataTest.tolist()  # on en fait une liste pour les besoins de l'exemple
print(dataTest)

[[0.04233015 0.98465436 0.11687597]
 [0.23556844 0.64372906 0.70559537]
 [0.7630281  0.31409605 0.04663236]
 ...
 [0.33749523 0.67348204 0.12491942]
 [0.9503992  0.89055991 0.5008607 ]
 [0.33859453 0.3364985  0.84479945]]


Prenons une fonction un peu lourde à calculer pour comparer deux approches: l'une itérative, l'autre vectorisée

$f(x,y,z)= \frac{x^{2}-\sqrt{y}}{\sqrt{x+y}} \times z$

In [4]:
resultsList = [] # on crée une liste vide
start = time() # on lance le chrono
for i in dataTestAsList:     # pour chaque ligne
    #result = (i[0]**-math.sqrt(i[1])) *i[2] / math.sqrt(i[0]+i[1])
    result = (i[0]**2-math.sqrt(i[1]))/math.sqrt(i[0]+i[1])*i[2]
    resultsList.append(result)

end = time() # on arrête le chrono

print("temps nécessaire: ",end - start)
print(resultsList[:15])          # on imprime les 15 premiers résultats
print(len(resultsList))

temps nécessaire:  0.8130707740783691
[-0.11423527993382593, -0.5619689797534317, 0.000978128863573517, -0.3067454589218615, -0.2289811936387017, -0.06242329610724, -0.14181208988963495, -0.009474164723078802, -0.04004719281138601, -0.03135133504588271, 0.17353468771358765, -0.31355334716551664, -0.2860803653087626, -0.39253659523073847, -0.09664010398628255]
600000


In [5]:
x = dataTest[:,0] # premier vecteur = première colonne de la matrice
y = dataTest[:,1] # second vecteur = seconde colonne de la matrice
z = dataTest[:,2]

start = time() # on lance le chrono
results = np.multiply(np.divide(np.power(x,2) - np.sqrt(y) ,np.sqrt(x+y) ),z)
end = time() # on arrête le chrono
print("temps nécessaire: ",end - start)
print(results)
print(len(results))

temps nécessaire:  0.044678449630737305
[-0.11423528 -0.56196898  0.00097813 ... -0.08780699 -0.01492674
 -0.47855764]
600000


On voit assez aisément que, même pour ce petit dataset de trois valeurs x y z de 600000 points de donnée, l'<b>approche iterative</b> par les bloucles prend plus de 10 fois plus de temps que l'<b>approche vectorielle</b> permise par numpy pour le <b>même calcul</b>. Il existe une correpondance intuitive entre:
<ul>
<li> $a*b$ et np.multiply(a,b)
<li> $a**2$ et np.power(x,2)
<li> $a+b$ et np.sum((x,y), axis=0)
</ul>

Toutefois, lorsque a et b peuvent s'apparenter à des <b>vecteurs</b> de même longueur, du point de vue de l'interpréteur, cela fait une ÉNORME différence de réaliser les opérations de manière vectorielle plutôt qu'élément par élément (<i>element-wise</i>). La leçon a retenir est double: 
<ul>
<li> aussi souvent que possible, si vous le pouvez, <b>évitez de recourir à des boucles lorsque vous avez des données structurées</b>
<li> ce n'est pas parceque vous avec l'impression de décomposer une opération dans votre code que vous facilitez la tâche de l'interpréteur (il y a une couche d'abstraction entre son monde et le vôtre)
</ul>

Nous verrons en outre que la vectorisation n'est pas le seul moyen d'optimiser un code

## De l'importance des contextes

Une autre habitude idiomatique à prendre en python consiste à recourir aux contextes (**with**). Classiquement, pour ouvrir un fichier et écrire sur ce dernier, nous dirions plus volontiers

In [ ]:
with open('test.txt', 'w') as f: 
    f.write('Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.')

Pourquoi recourrir aux contextes:
- dans un soucis d'efficience: l'objet f n'est en mémoire que durant l'exécution de ce context (la RAM est une ressource précieuse)
- dans un soucis de "propreté": le fichier est ici systématiquement fermé à la fin de la méthode write sans que nous aillons eu à l'expliciter.

Appliquons cela à une classe d'objet crée par nos soins. Concevons un simple objet qui scanne le contenu d'un dossier à intervalle régulier à la recherche de nouveaux fichiers. (un genre de "what just happend during this period")

In [ ]:
import os
import time

class DirWatcher():
    def __init__(self,path,files=[],new_files = []):
        self.path = path
        #self.delay = delay
        self.files = os.listdir(path)
        self.new_files = new_files
        print('initial files: ', self.files)    
    
    def __enter__(self):   
        pass
                                   
    def __exit__(self, exc_type, exc_value, tb):
        for f in os.listdir(self.path):
            if f not in self.files:
                self.new_files.append(f)
        print('new files: ', self.new_files)
          

In [ ]:
with DirWatcher('/home/david/Documents') as a:
    time.sleep(10)
    

## It is Easier to Ask for Forgiveness than Permission

Comprenez par là que le code en python fonctionne avec le postulat que les variables, fichiers, paths, etc nécessaires à l'exécution existent (Meinung, sort de ce code!!!!). D'où deux questions: 

- qu'en est-il lorsque tel n'est pas le cas ?
- comment distinguer les cas grave (nécessitant de stoper le code) des cas benins?

C'est ici que les structures en **try** ....**except** interviennent. Comparons:

In [ ]:
import os

if os.path.exists("file.txt"):
    os.unlink("file.txt") # supprime le fichier

notez que le fichier file.txt n'existe pas dans le répertoire courant. Pourtant, par la seule exécution du code, nous n'aurions jamais pu nous en rendre compte. Pour cette raison, il convient de partir du principe que le fichier existe et traiter tout problème comme une exception

In [ ]:
try:
    os.unlink("file.txt")
# raised when file does not exist
except OSError:
    print("file does not exist")
    pass

## Du besoin d'erreurs claires


blablablabla

In [ ]:
try: 
    open("file.txt")
except: 
    print("file does not exist")

In [ ]:
open("file.txt")

In [ ]:
try: 
    open("notebook.tex")
    var = non_existent_var
except FileNotFoundError: # voila comment distinguer les erreurs par type
    print("file does not exist")